In [14]:
# Memory
# Conversation Buffer Memory
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({"input":"Hey~!"}, {"output":"How was it going?"})
memory.load_memory_variables({})

{'history': 'Human: Hey~!\nAI: How was it going?'}

In [15]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input":"Hey~!"}, {"output":"How was it going?"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hey~!'),
  AIMessage(content='How was it going?')]}

In [17]:
# Conversation Buffer Window Memory
# 대화의 일부! 그 중에서 최근의 메세지를 저장하는 기능 - k 값에 따라 조절 가능  
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4,
)

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})

add_message(1,1)
add_message(2,2)
add_message(3,3)
add_message(4,4)
add_message(5,5)

memory.load_memory_variables({})



{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

In [18]:
# Conversation Summary Memory -> llm을 사용함! 
# 대화를 요약해서 정보를 저장하는 기능
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

add_message("South Kddorea is so pretty", "I wish I could go!!!")

get_history()


{'history': 'Nicolas from South Korea introduces himself to the AI, who responds with enthusiasm, finding it cool. The human mentions how pretty South Korea is, and the AI expresses a wish to visit.'}

In [19]:
# Conversation Summary Buffer Memory
# Conversation Summary Memory + Window Buffer Memory + Buffer Window Memory
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")


In [20]:
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is so cool!')]}

In [21]:
add_message("South Korea is so pretty", "I wish I could go!!!")

In [22]:
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!')]}

In [23]:
add_message("How far is Korea from Argentina?", "I don't know! Super far!")

In [24]:
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!")]}

In [25]:
add_message("How far is Brazil from Argentina?", "I don't know! Super far!")

In [26]:
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!")]}

In [27]:
# Conversation Knowledge Grahp Memory
# 오~! 이거 쫌 신기한듯! Chat 정보를 저장함
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

In [28]:
memory.load_memory_variables({"input": "who is Nicolas"})

{'history': [SystemMessage(content='On Nicolas: Nicolas lives in South Korea.')]}

In [29]:
add_message("Nicolas likes kimchi", "Wow that is so cool!")

In [30]:
memory.load_memory_variables({"inputs": "what does nicolas like"})

{'history': [SystemMessage(content='On Nicolas: Nicolas lives in South Korea. Nicolas likes kimchi.')]}

In [31]:
# 메모리를 체인에 연결하는 방법! 
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
)

template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True, #프롬프트의 로그를 확인할 수 있음! 
)

chain.predict(question="My name is Nico")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    
    Human:My name is Nico
    You:


> Finished chain.


'Hello Nico! How can I assist you today?'

In [34]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is Nico
AI: Hello Nico! How can I assist you today?
Human: I live in Seoul
AI: That's great to know! How can I assist you with information or tasks related to Seoul?
Human: What is my name?
AI: Your name is Nico.
    Human:I live in Seoul
    You:


> Finished chain.


'Yes, you mentioned that earlier. Is there anything specific you would like to know or discuss about living in Seoul?'

In [35]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is Nico
AI: Hello Nico! How can I assist you today?
Human: I live in Seoul
AI: That's great to know! How can I assist you with information or tasks related to Seoul?
Human: What is my name?
AI: Your name is Nico.
Human: I live in Seoul
AI: Yes, you mentioned that earlier. Is there anything specific you would like to know or discuss about living in Seoul?
    Human:What is my name?
    You:


> Finished chain.


'Your name is Nico. How can I assist you further today?'

In [36]:
# 기존 memory 경우 chat 형태로 결과값을 출력하지만 
"""
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
)
"""
# 아래 처럼 str 으로 결과를 출력할 수 있음

memory.load_memory_variables({})

{'chat_history': "System: The human introduces themselves as Nico. The AI greets Nico and asks how it can assist. Nico mentions they live in Seoul.\nAI: That's great to know! How can I assist you with information or tasks related to Seoul?\nHuman: What is my name?\nAI: Your name is Nico.\nHuman: I live in Seoul\nAI: Yes, you mentioned that earlier. Is there anything specific you would like to know or discuss about living in Seoul?\nHuman: What is my name?\nAI: Your name is Nico. How can I assist you further today?"}

In [37]:
#대화를 기반으로 채팅을 출력? 해서 사용할 수 있음
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder 

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True, #사람, AI 메세지를 실제 메세지 클래스로 변경하는 것!!!! 
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

chain.predict(question="My name is Nico")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Nico

> Finished chain.


'Nice to meet you, Nico! How can I assist you today?'

In [38]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Nico
AI: Nice to meet you, Nico! How can I assist you today?
Human: I live in Seoul

> Finished chain.


'Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?'

In [39]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Nico
AI: Nice to meet you, Nico! How can I assist you today?
Human: I live in Seoul
AI: Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?
Human: What is my name?

> Finished chain.


'Your name is Nico.'

In [40]:
chain.predict(question="대화내용 요약해줘?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Nico
AI: Nice to meet you, Nico! How can I assist you today?
Human: I live in Seoul
AI: Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?
Human: What is my name?
AI: Your name is Nico.
Human: 대화내용 요약해줘?

> Finished chain.


'당신의 이름은 Nico이고, 서울에 거주하고 계신 것 같아요. 무엇을 도와드릴까요?'

In [41]:
# off the shell -> memory | langchain expression languege -> LCEL
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    # memory_key="chat_history",    
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"), # 메모리 히스토리를 옮겨옴
        ("human", "{question}"),
    ]
)

# chain = LLMChain(
#     llm=llm,
#     memory=memory,
#     prompt=prompt,
#     verbose=True,
# )

# chain.predict(question="My name is Nico")

def load_memory(_):
    return memory.load_memory_variables({})["history"]

# Runnable Pass through
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)



# 코드 분석
"""
필요한 라이브러리와 클래스를 가져옵니다.
ChatOpenAI 인스턴스를 생성
낮은 temperature 값(0.1)은 일관된 출력을 위한 것
ConversationSummaryBufferMemory 설정

토큰 제한을 120으로 설정하여 메모리 사용 제어
대화 내용 요약 저장

ChatPromptTemplate 정의

시스템 메시지, 
대화 기록, 
사용자 질문을 포함

load_memory 함수는 메모리에서 대화 기록 로딩

RunnablePassthrough를 사용하여 체인 구성
- 대화 기록을 불러오고, 프롬프트를 적용한 후 LLM에 전달합니다.

invoke_chain 함수는 질문을 처리하고 결과를 메모리에 저장

이 코드를 더 복잡하고 강력한 시스템으로 발전시킬 수 있으며, 
예를 들어, 다중 에이전트 시스템, 도구 사용, 웹 검색 기능 등을 추가할 수 있음
    """

In [42]:
invoke_chain("My name is nico")

content='Hello Nico! How can I assist you today?'


In [43]:
invoke_chain("What is my name?")

content='Your name is Nico.'


In [ ]:
# 정리 
# 필요한 라이브러리와 클래스를 가져옵니다.
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# ChatOpenAI 인스턴스를 생성합니다.
# 낮은 temperature 값(0.1)은 일관된 출력을 위한 것입니다.
llm = ChatOpenAI(temperature=0.1)

# ConversationSummaryBufferMemory를 설정합니다.
# 토큰 제한을 120으로 설정하여 메모리 사용을 제어합니다.
# 대화 내용을 요약하여 저장합니다.
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

# ChatPromptTemplate을 정의합니다.
# 시스템 메시지, 대화 기록, 사용자 질문을 포함합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),  # 메모리 히스토리를 옮겨옴
        ("human", "{question}"),
    ]
)

# load_memory 함수는 메모리에서 대화 기록을 불러옵니다.
def load_memory(_):
    return memory.load_memory_variables({})["history"]

# RunnablePassthrough를 사용하여 체인을 구성합니다.
# 대화 기록을 불러오고, 프롬프트를 적용한 후 LLM에 전달합니다.
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

# invoke_chain 함수는 질문을 처리하고 결과를 메모리에 저장합니다.
def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

# 이 코드를 더 복잡하고 강력한 시스템으로 발전시킬 수 있으며,
# 예를 들어, 다중 에이전트 시스템, 도구 사용, 웹 검색 기능 등을 추가할 수 있습니다.

In [9]:
# 새로운 버전 
"""
이 시스템은 사용자의 입력에 응답하고 대화 내용을 요약하여 기억합니다. 
코드의 주요 구성 요소와 작동 방식은 다음과 같습니다:
1. ChatOpenAI 모델 초기화:
- `ChatOpenAI(temperature=0.7)`를 사용하여 AI 모델을 초기화합니다.

2. 메모리 설정:
- `ConversationSummaryMemory`를 사용하여 대화 내용을 요약하고 기억합니다.

3. 프롬프트 템플릿 정의:
- 시스템 메시지, 대화 기록, 사용자 입력을 포함하는 템플릿을 정의합니다.

4. ConversationChain 생성:
- 모델, 메모리, 프롬프트를 결합하여 대화 체인을 생성합니다.

5. chat_with_bot 함수:
- 사용자 입력을 받아 AI의 응답을 생성하고 출력합니다.

코드 실행 결과를 보면, 시스템이 3번 작동한 것으로 보입니다:

1. "안녕하세요! 오늘 기분이 어떠세요?"
2. "AI의 발전에 대해 어떻게 생각하시나요?"
3. "제가 아까 무엇에 대해 물어봤었나요?"

각 대화마다 "Entering new ConversationChain chain..."과 "Finished chain." 
메시지가 표시되어 있어, 대화 체인이 3번 실행되었음을 알 수 있습니다.

마지막으로, 대화 요약이 출력되었습니다. 이는 `memory.load_memory_variables({})`를 통해 저장된 대화 내용을 불러온 것입니다.

이 시스템의 장점은 대화 내용을 요약하여 기억하기 때문에, 이전 대화 맥락을 유지하면서 응답할 수 있다는 것입니다. 
예를 들어, 세 번째 질문에서 AI가 이전에 무엇에 대해 물어봤는지 정확히 기억하고 있는 것을 볼 수 있습니다.
"""
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)

# ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(temperature=0.7)

# 메모리 설정
memory = ConversationSummaryMemory(llm=llm, return_messages=True)

# 프롬프트 템플릿 정의
system_message = "당신은 친절하고 도움이 되는 AI 어시스턴트입니다. 사용자와의 대화를 기억하고 이전 대화 내용을 바탕으로 응답합니다."
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_message),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

# 대화 체인 생성
conversation = ConversationChain(
    llm=llm, 
    memory=memory,
    prompt=prompt,
    verbose=True
)

def chat_with_bot(input_text):
    response = conversation.predict(input=input_text)
    print(f"Human: {input_text}")
    print(f"AI: {response}\n")

# 사용 예시
chat_with_bot("안녕하세요! 오늘 기분이 어떠세요?")
chat_with_bot("AI의 발전에 대해 어떻게 생각하시나요?")
chat_with_bot("제가 아까 무엇에 대해 물어봤었나요?")

# 대화 요약 출력
print("대화 요약:")
print(memory.load_memory_variables({})["history"])



> Entering new ConversationChain chain...
Prompt after formatting:
System: 당신은 친절하고 도움이 되는 AI 어시스턴트입니다. 사용자와의 대화를 기억하고 이전 대화 내용을 바탕으로 응답합니다.
System: 
Human: 안녕하세요! 오늘 기분이 어떠세요?

> Finished chain.
Human: 안녕하세요! 오늘 기분이 어떠세요?
AI: 안녕하세요! 제 기분은 항상 좋습니다. 여러분의 하루가 행복하고 유익하길 바라요. 혹시 오늘 무슨 일을 하셨나요?



> Entering new ConversationChain chain...
Prompt after formatting:
System: 당신은 친절하고 도움이 되는 AI 어시스턴트입니다. 사용자와의 대화를 기억하고 이전 대화 내용을 바탕으로 응답합니다.
System: The human greets the AI in Korean and asks how it's feeling. The AI responds that it always feels good and wishes everyone a happy and productive day. The AI then asks the human what they did today.
Human: AI의 발전에 대해 어떻게 생각하시나요?

> Finished chain.
Human: AI의 발전에 대해 어떻게 생각하시나요?
AI: AI의 발전에 대해 저는 매우 긍정적으로 생각합니다. AI 기술의 발전으로 우리는 더 빠르고 효율적으로 일을 처리할 수 있고, 새로운 혁신적인 서비스와 제품을 경험할 수 있게 됩니다. 물론 이러한 발전은 윤리적인 문제와 함께 고민해야 할 부분도 많지만, 적절히 활용된다면 인류에 많은 혜택을 줄 수 있다고 생각합니다. 혹시 당신은 AI의 발전에 대해 어떻게 생각하시나요?



> Entering new ConversationChain chain...
Prompt after formatt

In [10]:
# 과제
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=1) # 창의력을 높이기 위해 적정 값 1 적용

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120, 
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 질문에 대한 답변을 이모지 3개만을 활용해서 만들어 낼거야"),
        MessagesPlaceholder(variable_name="history"),  # 메모리 히스토리를 옮겨옴
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)
    

invoke_chain("트랜스포머")
invoke_chain("매트릭스")
invoke_chain("라이언일병구하기")	
print(memory.load_memory_variables({})["history"])

content='🤖🚗🔥'
content='💊💥🔵'
content='🦁💪👊'
[HumanMessage(content='트랜스포머'), AIMessage(content='🤖🚗🔥'), HumanMessage(content='매트릭스'), AIMessage(content='💊💥🔵'), HumanMessage(content='라이언일병구하기'), AIMessage(content='🦁💪👊')]
